In [1]:
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt


In [22]:
def F_nu(t_p,nu_p,F_nup):
    
    m = 0.88 #using stAandard model
    p = 3.0 #using standard model
    a = (2*m)+0.5
    b = (p+5-(6*m))/2
    t=t_p
    nu = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])#nu_p
    Fnu = F_nup*1.582*(t/t_p)**a*(nu/nu_p)**(5/2)*(1-np.exp(-(t/t_p)**(-(a+b))*(nu/nu_p)**(-(p+4)/2)))
    
    return nu,Fnu
                                                    
#freqs,SED = F_nu(1224.288326 , 8,  0.0272)   
#print(freqs)
#print(SED)

def SSA_props(t_p,nu_p,F_nup,D):
    '''Calculates Synchroton Self-Absorption properties
    t_p: time of peak in days
    nu_p: frequence of peak in GHz
    F_nup: peak flux in mJy
    D: distance to SN in Mpc'''
    
    f =  0.5   # filling factor (fraction of the emiting region)
    alpha = 1  #fraction of charged particles and magnetic field (epsilon_e/epsilon_b)
    v_w = 1000  #wind speed in km/s
    epsilon_b = 0.1

    #Radius
    R = 4.0e14*(alpha)**(-1/19)*(f/0.5)**(-1/19)*(F_nup)**(9/19)*(D)**(18/19)*(nu_p/5)**(-1)

    # Magnetic field flux
    B = 1.1*(alpha)**(-4/19)*(f/0.5)**(-4/19)*(F_nup)**(-2/19)*(D)**(-4/19)*(nu_p/5)

    #Internal energy of the emitting material
    E = (1/epsilon_b)*((B**2)/(8* 3.142))*((4*3.142*f*R**3)/3)

    #expansion velocity in km/s
    v = (R/t_p)*1.1574e-10

    #pre-explosion mass-loss in 1e-5 solar mass per year
    M = 1.0*(alpha)**(-8/19)*epsilon_b*(f)**(-8/19)*(F_nup)**(-4/19)*(D)**(-4/19)*(nu_p/5)*(t_p/10)*(v_w/1000)

    return R,B,E,v,M


def taufreefree(M,R,nu):

    Z_ave = 5.4  # Average metallicity 1= pure H. 5.4 for a massive star
    miu = 1.9 # mean molecular weight of electrons. 1= pure H  1.9 is for a massive star.
    vw_cgs = 1000 * 1e5 # assumed wind velocity in cgs (cm/s). Take this as 1000 * 10^5 for now. (i.e. 1000 km/s in cgs)
    T = 10^4 # temperature of the material absorbing in K.  10^4 is a good starting point. 
        
    M_cgs =  M * 1e-5 * 6.307e+25 #mass loss rate in cgs units
    
    tau_ff = 2.021e25*M_cgs**2*Z_ave/(miu**2*nu**(2.1)*R**3*vw_cgs**2*T**(1.35))  
    
    return tau_ff


freqs,SED = F_nu(1224.288326 , 8,  0.0272)       
R2,B2,E2,v2,M2 = SSA_props(1224.288326 , 8,  0.0272,880) 
tauff=taufreefree(M2,R2,freqs)
#tauff=taufreefree(4,3e14,np.array([2,5,7]))
print(freqs)
print(tauff)
print(R2,B2,E2,v2,M2)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[283.06682158  66.02767084  28.17956817  15.40149881   9.63944344
   6.5731167    4.75535654   3.59252663   2.8053025    2.24847969
   1.84062082   1.53323368   1.29600873   1.10922613   0.95961566]
2.7920932311552916e+16 0.6171685679345864 6.909014440834395e+48 2639.548737916443 13.440843218384842


In [24]:
F_p = [0.01,0.015,0.02,0.025,0.03] #mJy
nu_p = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15] # GHz
t_p = 1224.288 # days
D = 808.4 # Mpc

for F in F_p:
    for nu in nu_p:
        #calculate SSA
        SSA_freq,SSA_sed = F_nu(t_p,nu,F)
        
        #calculate Mass loss rate, Radius, velocity, etc.
        R,B,E,v,M = SSA_props(t_p,nu,F,D)
        
        #calculate tau_ff
        tauff = taufreefree(M,R,SSA_freq)
        
        #Correct SED for FFA:
        SSA_FFA_sed = SSA_sed*np.exp(-tauff)
        
        #Evaluate if this SED is allowed by your data.
        
        #Need to known what frequency your data point was
        #Need to know your upper limit.
        
        #**Need to interpolate SSA_FFA_sed to the frequency of your observation.
        
        
        #Figure out if the flux there is above (disallowed) or below (allowed) your limit.
        #print out to a file F,nu, R,B,E,v,M, and whether it is ruled out.